# QOSF Screening Task 4

### Task

Find the lowest eigenvalue of the following matrix:

[1 0 0 0;
0 0 -1 0;
0 -1 0 0;
0 0 0 1]

using VQE-like circuits, created by yourself from scratch.

### Solution

For the solution, I have referred to the following paper by P.J.J O'Malley et al.
[Scalable Quantum Simulation of Molecular Energies](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.6.031007)

A high level sketch of the VQE circuit I prepared is provided, which is also from O'Malley's research paper.
![image](https://github.com/abhishekjay/VQE-QOSF-Task4/blob/master/assets/vqe-circuitplan.PNG?raw=true)

#### Importing necessary packages

In [1]:
import numpy as np
from scipy.linalg import block_diag
from scipy.optimize import minimize

#### Let us define the matrices of the gates to be used. 

Pauli matrices

In [2]:
I = np.array([[1, 0],
              [0, 1]])

PauliX = np.array([[0, 1],
                   [1, 0]])

PauliY = np.array([[0,-1j],
                   [1j, 0]])

PauliZ = np.array([[1, 0],
                   [0,-1]])

Hadamard, Phase and Rotational gates.

In [3]:
# Hadamard gate
H = (1/np.sqrt(2))*np.array([[ 1, 1],
                             [ 1,-1]])

# Phase matrix
S = np.array([[ 1, 0],
              [ 0,1j]])

# single qubit basis states |0> and |1>
q0 = np.array([[1],
               [0]])
q1 = np.array([[0],
               [1]])

# Projection matrices |0><0| and |1><1|
P0  = np.dot(q0,q0.conj().T)
P1  = np.dot(q1,q1.conj().T)


# Rotation matrices Rx, Ry and Rz
Rx = lambda theta : np.array([[    np.cos(theta/2),-1j*np.sin(theta/2)],
                              [-1j*np.sin(theta/2),    np.cos(theta/2)]])
Ry = lambda theta : np.array([[    np.cos(theta/2),   -np.sin(theta/2)],
                              [    np.sin(theta/2),    np.cos(theta/2)]])
Rz = lambda theta : np.array([[np.exp(-1j*theta/2),                0.0],
                              [                0.0, np.exp(1j*theta/2)]])

Entanglement and SWAP gates

In [4]:
# CNOTij, where i is control qubit and j is target qubit
CNOT10 = np.array([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,0,1],
                   [0,0,1,0]])

CNOT01 = np.array([[1,0,0,0],
                   [0,0,0,1],
                   [0,0,1,0],
                   [0,1,0,0]])

SWAP   = np.array([[1,0,0,0],
                   [0,0,1,0],
                   [0,1,0,0],
                   [0,0,0,1]])

#### Next, we define the function for decomposing the Hamiltonian into Pauli matrices.

In [5]:
#Hilbert-Schmidt-Product of two matrices M1, M2
def HS(M1, M2):
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def decompose(H):   
    S = [I, PauliX, PauliY, PauliZ]
    labels = ['I', 'Pauli_x', 'Pauli_y', 'Pauli_z']
    for i in range(4):
        for j in range(4):
            label = labels[i] + ' \otimes ' + labels[j]
            a_ij = 0.25 * HS(np.kron(S[i], S[j]), H)
            if a_ij != 0.0:
                print(a_ij,"    *    ", label)

In [6]:
Hamiltonian = np.array([[1,0,0,0],[0,0,-1,0],[0,-1,0,0],[0,0,0,1]], dtype=np.complex128)
decompose(Hamiltonian)

(0.5+0j)     *     I \otimes I
(-0.5+0j)     *     Pauli_x \otimes Pauli_x
(-0.5+0j)     *     Pauli_y \otimes Pauli_y
(0.5+0j)     *     Pauli_z \otimes Pauli_z


#### We have decomposed the Hamiltonian and obtained the coefficients. 

In [7]:
g0 = 0.5      # coefficient of I tensor I
g1 = +0.5     # coefficient of Pauli Z tensor Pauli Z
g2 = -0.5     # coefficient of Pauli Y tensor Pauli Y
g3 = -0.5     # coefficient of Pauli X tensor Pauli X

The classical minimum expectation energy of the Hamiltonian can be calculated by taking the first entry from np.linalg.eigvalsh(Hamiltonian).

In [8]:
classical_energy = np.linalg.eigvalsh(Hamiltonian)[0] # taking the lowest value
print("Classical diagonalization energy: ", classical_energy)

Classical diagonalization energy:  -1.0


Now let's define the initial state of the qubits. We take the intial state as |01>. 

In [9]:
# initializing basis state as |01> 
psi0 = np.zeros((4,1))
psi0[1] = 1
print(psi0)

[[0.]
 [1.]
 [0.]
 [0.]]


#### For the ansatz, we choose the following quantum circuit. The blue line shows qubit 1 and the red line qubit 0. RZ is the rotational gate with variational parameter theta.

![image](https://github.com/abhishekjay/VQE-QOSF-Task4/blob/master/assets/vqe-ansatz.PNG?raw=true)

In [10]:
# the ansatz is read from right to left
ansatz = lambda theta: (np.dot(np.dot(np.kron(-Ry(np.pi/2),Rx(np.pi/2)),
                                      np.dot(CNOT10, 
                                             np.dot(np.kron(I,Rz(theta)),
                                                    CNOT10))),
                               np.kron(Ry(np.pi/2),-Rx(np.pi/2))))

#### We can now define the projective measurements for each decomposition. 

In [11]:
def projective(theta,ansatz,psi0):
    circuit = ansatz(theta)
    psi = np.dot(circuit,psi0)
    
    measureZ = lambda U: np.dot(np.conj(U).T,np.dot(np.kron(PauliZ,I),U))   
    
    energy = 0.0
     
    energy += g0

    # <PauliZ1 PauliZ0>
    U = CNOT01
    energy += g1*np.dot(psi.conj().T,np.dot(measureZ(U),psi))
    
    # <PauliY1 PauliY0>
    U = np.dot(CNOT01,np.kron(np.dot(H,S.conj().T),np.dot(H,S.conj().T)))
    energy += g2*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    # <PauliX1 PauliX0>
    U = np.dot(CNOT01,np.kron(H,H))
    energy += g3*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    return np.real(energy)[0,0]

#### Finally, we can estimate the lowest eigenstate of the hamiltonian using VQE, by minimizing the energy.

In [12]:
theta  = np.linspace(0, 2*np.pi, 361)
result = [0.0]
min_energy = 0
theta_final = 0
#result = minimize(projective,theta,args=(ansatz,psi0))
for i in range(1,361):
    energy = projective(theta[i], ansatz, psi0)
    result.append(energy)
    print(energy)
    if(result[i]<result[i-1]):
        min_energy = result[i]
        theta_final = theta[i]
print("VQE: ")
print("theta: ", theta_final, " rad")
print("energy: ", min_energy, " Eh")

0.017452406437283075
0.03489949670250054
0.05233595624294342
0.06975647374412489
0.08715574274765765
0.1045284632676532
0.12186934340514709
0.13917310096006505
0.15643446504023045
0.17364817766692994
0.19080899537654436
0.207911690817759
0.22495105434386464
0.24192189559966737
0.2588190451025204
0.2756373558169988
0.29237170472273644
0.3090169943749471
0.32556815445715637
0.3420201433256683
0.3583679495453
0.3746065934159118
0.39073112848927355
0.40673664307580004
0.4226182617406992
0.4383711467890772
0.4539904997395465
0.4694715627858906
0.48480962024633684
0.4999999999999998
0.5150380749100539
0.5299192642332047
0.544639035015027
0.5591929034707467
0.5735764363510458
0.5877852522924729
0.601815023152048
0.6156614753256581
0.6293203910498372
0.642787609686539
0.6560590289905069
0.669130606358858
0.6819983600624984
0.694658370458997
0.7071067811865475
0.7193398003386511
0.7313537016191705
0.7431448254773942
0.7547095802227719
0.766044443118978
0.7771459614569709
0.7880107536067218
0.79

#### The energy values obtained from classical solution and VQE match.